In [1]:
!pip install -q transformers accelerate datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 114.0 MB/s eta 0:00:00


In [2]:
! pip install -U "huggingface_hub[cli]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.2
    Uninstalling huggingface-hub-0.33.2:
      Successfully uninstalled huggingface-hub-0.33.2


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
import random
import pandas as pd
import torch
import os
from huggingface_hub import login
login("")


In [4]:
MODEL_NAME =  "TinyLlama/TinyLlama-1.1B-Chat-v1.0" #"mistralai/Mistral-7B-Instruct-v0.1"#"tiiuae/falcon-rw-1b"#"google/flan-t5-large"# "google/flan-t5-base"#"mistralai/Mistral-7B-Instruct-v0.1"  # or "meta-llama/Meta-Llama-3-8B-Instruct"

In [5]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Pipeline for generation
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda:0


In [12]:
import json
from copy import deepcopy


input_json = {
    "claims": [
        {"text": "Social media negatively impacts mental health."},
        {"text": "Renewable energy is essential for combating climate change."},
        {"text": "Online education can be as effective as traditional classroom learning."},
        {"text": "Banning plastic straws won't significantly reduce ocean pollution."}
    ],
    "premises": [
        {"text": "Studies have shown increased anxiety and depression in heavy social media users."},
        {"text": "Social media creates unrealistic expectations through curated content."},
        {"text": "For many users, social media serves as a vital connection to friends and support groups."},
        {"text": "Solar and wind energy produce no direct carbon emissions."},
        {"text": "Transitioning to renewable energy reduces dependence on fossil fuels."},
        {"text": "Solar panel production and disposal generate toxic waste."},
        {"text": "Renewable energy sources are less reliable due to weather dependency."},
        {"text": "Virtual classrooms allow students to access materials at their own pace."},
        {"text": "Online platforms offer interactive tools and instant feedback."},
        {"text": "Lack of face-to-face interaction can hinder communication skills."},
        {"text": "Students in online programs often struggle with motivation and time management."},
        {"text": "Plastic straws make up less than 1% of ocean plastic waste."},
        {"text": "Efforts should focus on larger polluters like fishing gear and plastic packaging."},
        {"text": "Plastic straw bans raise awareness about single-use plastics."},
        {"text": "Alternatives like paper straws often perform worse and are not always recyclable."}
    ]
}


TINYLLAMA_PROMPT = lambda claim, premise: f"""You are given a claim and evidence. Determine whether the evidence supports(pro) or refutes(con) the claim.
Respond only with one word: "pro" or "con".

Claim: {claim}
Evidence: {premise}
Stance:"""


output_json = deepcopy(input_json)
for premise_obj in output_json["premises"]:
    premise_text = premise_obj["text"]
    # You can choose how to aggregate stances across claims: here we do majority vote
    stance_votes = []
    for claim_obj in input_json["claims"]:
        claim_text = claim_obj["text"]
        prompt = TINYLLAMA_PROMPT(claim_text, premise_text)

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=10)
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).lower().strip()

        # Normalization
        if "refute" in decoded:
            stance_votes.append("con")
        elif "support" in decoded:
            stance_votes.append("pro")

    # Majority vote (you can choose first or more advanced logic)
    if stance_votes:
        final_stance = max(set(stance_votes), key=stance_votes.count)
    else:
        final_stance = "unknown"

    premise_obj["stance"] = final_stance

# ✅ Final output
print(json.dumps(output_json, indent=2))


{
  "claims": [
    {
      "text": "Social media negatively impacts mental health."
    },
    {
      "text": "Renewable energy is essential for combating climate change."
    },
    {
      "text": "Online education can be as effective as traditional classroom learning."
    },
    {
      "text": "Banning plastic straws won't significantly reduce ocean pollution."
    }
  ],
  "premises": [
    {
      "text": "Studies have shown increased anxiety and depression in heavy social media users.",
      "stance": "con"
    },
    {
      "text": "Social media creates unrealistic expectations through curated content.",
      "stance": "con"
    },
    {
      "text": "For many users, social media serves as a vital connection to friends and support groups.",
      "stance": "con"
    },
    {
      "text": "Solar and wind energy produce no direct carbon emissions.",
      "stance": "con"
    },
    {
      "text": "Transitioning to renewable energy reduces dependence on fossil fuels.",
  